In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [2]:
import nltk
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


True

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.  

In [0]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)

alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [0]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [6]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [0]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
   
# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [8]:
list(common_words)[0]

'worthy'

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(vocabulary=common_words)
df = pd.DataFrame()
df['text_sentence'] = sentences[0]
df['text_source'] = sentences[1]
#String version of the spacy objects
df['str_sentence'] = [_.text for _ in df.text_sentence]

X = vectorizer.fit_transform(df['str_sentence'])

df2 =  pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

word_counts = pd.concat([df,df2], axis=1)
print(word_counts.shape)
word_counts.head()

(5318, 3065)


,text_sentence,text_source,str_sentence,'s,-PRON-,1,`,a,abide,ability,...,you,you'd,young,your,yours,youth,zeal,zealand,zealous,zigzag
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,Alice was beginning to get very tired of sitti...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(So, she, was, considering, in, her, own, mind...",Carroll,So she was considering in her own mind (as wel...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,There was nothing so VERY remarkable in that; ...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"(Oh, dear, !)",Carroll,Oh dear!,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(I, shall, be, late, !, ')",Carroll,I shall be late!',0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


 ## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [10]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source','str_sentence'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training set score: 0.9846394984326019

Test set score: 0.8472744360902256


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [11]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 3062) (3190,)
Training set score: 0.9545454545454546

Test set score: 0.8843984962406015


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [12]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8642633228840125

Test set score: 0.8529135338345865


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data:

In [13]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [0]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [0]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [17]:
emma_sentences = pd.DataFrame(emma_sents)
emma_sentences.head()

,0,1
0,"(Emma, Woodhouse, ,, handsome, ,, clever, ,, a...",Austen
1,"(She, was, the, youngest, of, the, two, daught...",Austen
2,"(Her, mother, had, died, too, long, ago, for, ...",Austen
3,"(Sixteen, years, had, Miss, Taylor, been, in, ...",Austen
4,"(Between, _, them)",Austen


In [0]:

emma = pd.DataFrame()
emma['text_sentence'] = emma_sentences[0]
emma['text_source'] = emma_sentences[1]
#String version of the spacy objects
emma['str_sentence'] = [_.text for _ in emma.text_sentence]

In [19]:
X = vectorizer.fit_transform(emma['str_sentence'])
df2 =  pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

emma_bow = pd.concat([emma,df2], axis=1)
print(emma_bow.shape)
emma_bow.head()


(1669, 3065)


,text_sentence,text_source,str_sentence,'s,-PRON-,1,`,a,abide,ability,...,you,you'd,young,your,yours,youth,zeal,zealand,zealous,zigzag
0,"(Emma, Woodhouse, ,, handsome, ,, clever, ,, a...",Austen,"Emma Woodhouse, handsome, clever, and rich, wi...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(She, was, the, youngest, of, the, two, daught...",Austen,She was the youngest of the two daughters of a...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(Her, mother, had, died, too, long, ago, for, ...",Austen,Her mother had died too long ago for her to ha...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"(Sixteen, years, had, Miss, Taylor, been, in, ...",Austen,Sixteen years had Miss Taylor been in Mr. Wood...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(Between, _, them)",Austen,Between _them,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source', 'str_sentence'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.6923937360178971


col_0,Austen,Carroll
row_0,,
Austen,1536,133
Carroll,692,321


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

In [0]:
# Adding in column for the number of words in each sentence
word_counts['sent_length'] = word_counts.text_sentence.map(lambda x: len(x))

sentences = word_counts.text_sentence

In [0]:
# Creating list of number of adverbs in each sentence
sentences = word_counts.text_sentence
adv_count = []
for sent in sentences:
    advs = 0
    for token in sent:
        if token.pos_ == 'ADV':
            advs +=1
    adv_count.append(advs)

In [0]:
# Creating column for number of adverbs per sentence
word_counts['adv_count'] = adv_count

# Creating list of number of verbs in each sentence
verb_count = []
for sent in sentences:
    verb = 0
    for token in sent:
        if token.pos_ == 'VERB':
            verb +=1
    verb_count.append(verb)

word_counts['verb_count'] = verb_count


In [0]:
# Creating list of number of nouns in each sentence
noun_count = []
for sent in sentences:
    noun = 0
    for token in sent:
        if token.pos_ == 'NOUN':
            noun +=1
    noun_count.append(noun)

word_counts['noun_count'] = noun_count

In [0]:
# Creating list of number of punctuation marks in each sentence
punc_count = []
for sent in sentences:
    punc = 0
    for token in sent:
        if token.pos_ == 'PUNCT':
            punc +=1
    punc_count.append(punc)

word_counts['punc_count'] = punc_count

In [26]:
word_counts.head()

,text_sentence,text_source,str_sentence,'s,-PRON-,1,`,a,abide,ability,...,youth,zeal,zealand,zealous,zigzag,sent_length,adv_count,verb_count,noun_count,punc_count
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,Alice was beginning to get very tired of sitti...,0,0,0,0,0,0,0,...,0,0,0,0,0,67,3,13,12,10
1,"(So, she, was, considering, in, her, own, mind...",Carroll,So she was considering in her own mind (as wel...,0,0,0,0,0,0,0,...,0,0,0,0,0,63,7,11,8,7
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,There was nothing so VERY remarkable in that; ...,0,0,0,0,0,0,0,...,0,0,0,0,0,33,6,5,2,4
3,"(Oh, dear, !)",Carroll,Oh dear!,0,0,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,1
4,"(I, shall, be, late, !, ')",Carroll,I shall be late!',0,0,0,0,0,0,0,...,0,0,0,0,0,6,0,2,0,2


In [28]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source','str_sentence'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 3067) (3190,)
Training set score: 0.9557993730407524

Test set score: 0.8900375939849624


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


We can see that the accuracy increased slightly using the number of words, verbs, adverbs, nouns and punctuations. 


# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [29]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


### Using Parents by Edgeworth.

In [31]:
#Can this differentiate Paradise from Alice in Wonderland?

parents = gutenberg.raw('edgeworth-parents.txt')
parents = re.sub(r'VOLUME \w+', '', parents)
parents = re.sub(r'CHAPTER \w+', '', parents)
parents = text_cleaner(parents)
print(type(parents))
print(parents[:100])

<class 'str'>
THE ORPHANS. Near the ruins of the castle of Rossmore, in Ireland, is a small cabin, in which there 


In [32]:
len(parents)

903807

In [33]:
len(alice)

141708

In [0]:
parentslen = md[0:len(alice)]
parents_doc = nlp(parentslen)

In [0]:
parents_sents = [[sent, "Edgeworth"] for sent in parents_doc.sents]
alice_sents = alice_sents[0:len(parents_sents)]

In [64]:
len(alice_sents)

1196

In [65]:
len(parents_sents)

1196

In [66]:
parents_sentences = pd.DataFrame(parents_sents)
parents_sentences.head()


,0,1
0,"(THE, ORPHANS, .)",Edgeworth
1,"(Near, the, ruins, of, the, castle, of, Rossmo...",Edgeworth
2,"(As, long, as, she, was, able, to, work, ,, sh...",Edgeworth
3,"(Mary, was, at, this, time, about, twelve, yea...",Edgeworth
4,"(One, evening, she, was, sitting, at, the, foo...",Edgeworth


In [0]:

parent = pd.DataFrame()
parent['text_sentence'] = parents_sentences[0]
parent['text_source'] = parents_sentences[1]
#String version of the spacy objects
parent['str_sentence'] = [_.text for _ in parent.text_sentence]

In [68]:
X = vectorizer.fit_transform(parent['str_sentence'])
df3 =  pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

parent_bow = pd.concat([parent,df3], axis=1)
print(parent_bow.shape)
parent_bow.head()


(1196, 3065)


,text_sentence,text_source,str_sentence,'s,-PRON-,1,`,a,abide,ability,...,you,you'd,young,your,yours,youth,zeal,zealand,zealous,zigzag
0,"(THE, ORPHANS, .)",Edgeworth,THE ORPHANS.,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(Near, the, ruins, of, the, castle, of, Rossmo...",Edgeworth,"Near the ruins of the castle of Rossmore, in I...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(As, long, as, she, was, able, to, work, ,, sh...",Edgeworth,"As long as she was able to work, she was very ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"(Mary, was, at, this, time, about, twelve, yea...",Edgeworth,Mary was at this time about twelve years old.,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(One, evening, she, was, sitting, at, the, foo...",Edgeworth,One evening she was sitting at the foot of her...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
# Identifying variables
X_parent = parent_bow.drop(['text_sentence','text_source','str_sentence'], 1)
y_parent = parent_bow.text_source

alice_wc = word_counts[word_counts.text_source == 'Carroll']
X_alice = alice_wc.drop(['text_sentence','text_source','str_sentence'], 1)
y_alice = alice_wc.text_source[1196:]

# Combine the Paradise sentence data with the Alice data from the test set.
X_pa = pd.concat([X_parent, X_alice], 0).dropna()
y_pa = pd.concat([y_parent, y_alice], 0).dropna()

# Split into train and test sets
X_train_pa, X_test_pa, y_train_pa, y_test_pa = train_test_split(X_pa, y_pa, test_size=0.4, random_state=0)

# Model
lr2 = LogisticRegression()
train = lr2.fit(X_train_pa, y_train_pa)
print('Train set score:', lr2.score(X_train_pa, y_train_pa))
print('\nTest set score:', lr2.score(X_test_pa, y_test_pa))
lr_pa_predicted = lr2.predict(X_test_pa)
pd.crosstab(y_test_pa, lr_pa_predicted)

Train set score: 0.9100899100899101

Test set score: 0.812874251497006


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


col_0,Carroll,Edgeworth
text_source,,
Carroll,89,104
Edgeworth,21,454


In [80]:
# Identifying variables
X_parent = parent_bow.drop(['text_sentence','text_source','str_sentence'], 1)
y_parent = parent_bow.text_source

persuasion_wc = word_counts[word_counts.text_source == 'Austen']
X_persuasion = persuasion_wc.drop(['text_sentence','text_source','str_sentence'], 1)
y_persuasion = persuasion_wc.text_source[1196:]

# Combine the Paradise sentence data with the Alice data from the test set.
X_pp = pd.concat([X_parent, X_persuasion], 0).dropna()
y_pp = pd.concat([y_parent, y_persuasion], 0).dropna()

# Split into train and test sets
X_train_pp, X_test_pp, y_train_pp, y_test_pp = train_test_split(X_pp, y_pp, test_size=0.4, random_state=0)

# Model
lr2 = LogisticRegression()
train = lr2.fit(X_train_pp, y_train_pp)
print('Train set score:', lr2.score(X_train_pp, y_train_pp))
print('\nTest set score:', lr2.score(X_test_pp, y_test_pp))
lr_pp_predicted = lr2.predict(X_test_pp)
pd.crosstab(y_test_pp, lr_pp_predicted)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Train set score: 0.8876199177706715

Test set score: 0.6821917808219178


col_0,Austen,Edgeworth
text_source,,
Austen,819,138
Edgeworth,326,177


Looks like the model did an ok job distinguishing the different models